<a href="https://colab.research.google.com/github/Amasha03/Customer_Churn_Prediction/blob/main/notebooks/02_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telco Customer Churn Prediction
### CM2604 Machine Learning CourseWork
02-Preprocessing  

Student Name : Amasha Widanagamage  
RGU ID : 2425800  
IIT ID : 20241246

In [18]:
#import libraries
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import joblib
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [19]:
#Load dataset
from google.colab import drive
df=pd.read_csv('/content/drive/MyDrive/ML/coursework/Telco-Customer-Churn.csv')
print("Original shape:", df.shape)
df.head()


Original shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [20]:
#Handle missing/ invalid values
df=df.replace(' ',np.nan)
df=df.dropna()
print("Shape after handling missing values:", df.shape)


#Convert TotalCharges to numeric
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')
df['TotalCharges']=df['TotalCharges'].fillna(df['TotalCharges'].median())


Shape after handling missing values: (7032, 21)


In [21]:
#Encode Churn
#Check what values exist
print("\nChurn values before encoding:")
print(df['Churn'].value_counts(dropna=False))


#Clean and map
df['Churn']=df['Churn'].str.strip().str.capitalize()
df['Churn']=df['Churn'].map({'Yes':1,'No':0})

#Check for unmapped values
print("\nChurn values after mapping:")
print(df['Churn'].value_counts(dropna=False))

#Handle NaN from unmapped values
if df['Churn'].isnull().any():
  df=df.dropna(subset=['Churn'])
  print(f"Dropped rows with NaN in churn.")

df['Churn']=df['Churn'].astype(int)


Churn values before encoding:
Churn
No     5163
Yes    1869
Name: count, dtype: int64

Churn values after mapping:
Churn
0    5163
1    1869
Name: count, dtype: int64


In [22]:
#Encoding binary column
binary_cols=['gender','Partner','Dependents','PhoneService','PaperlessBilling']
for col in binary_cols:
    df[col]=df[col].astype(str).str.strip().str.capitalize()
    df[col]=df[col].map({'Yes':1,'No':0,'Female':1,'Male':0})
    #Fill any NaN with mode
    if df[col].isnull().any():
      df[col].fillna(df[col].mode()[0])
    df[col]=df[col].astype(int)

In [23]:
#Scale numerical features
numerical_cols=['tenure','MonthlyCharges','TotalCharges']
scaler=StandardScaler()
df[numerical_cols]=scaler.fit_transform(df[numerical_cols])

In [24]:
#One-hot encode other categorical columns
categorical_cols=['MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaymentMethod']

for col in categorical_cols:
  df[col]=df[col].astype(str).str.strip()

df=pd.get_dummies(df,columns=categorical_cols,drop_first=True)



In [25]:
df.to_csv('/content/drive/MyDrive/ML/coursework/preprocessed_data.csv',index=False)
print("Dataset saved successfully!")

Dataset saved successfully!
